<a href="https://colab.research.google.com/github/adarsh567527/Speech-Emotion-Recognition-SER-/blob/main/SER_SVM_RF_KNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive/')


MessageError: ignored

In [ ]:
import librosa
import soundfile
import os, glob, pickle
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix

from sklearn.metrics import accuracy_score

In [ ]:
def extract_feature(file_name, mfcc, chroma, mel):
    X, sample_rate = librosa.load(os.path.join(file_name), res_type='kaiser_fast')
    if chroma:
        stft=np.abs(librosa.stft(X))
    result=np.array([])
    if mfcc:
        mfccs=np.mean(librosa.feature.mfcc(y=X, sr=sample_rate, n_mfcc=40).T, axis=0)
        result=np.hstack((result, mfccs))
    if chroma:
        chroma=np.mean(librosa.feature.chroma_stft(S=stft, sr=sample_rate).T,axis=0)
        result=np.hstack((result, chroma))
    if mel:
        mel=np.mean(librosa.feature.melspectrogram(X, sr=sample_rate).T,axis=0)
        result=np.hstack((result, mel))
    return result

In [ ]:
# Emotions in the RAVDESS & TESS dataset
emotions={
  '01':'neutral',
  '02':'calm',
  '03':'happy',
  '04':'sad',
  '05':'angry',
  '06':'fearful',
  '07':'disgust',
  '08':'surprised'
}
# Emotions to observe
observed_emotions=['neutral','calm','happy','sad','angry','fearful', 'disgust','surprised']

In [ ]:
def load_data(test_size=0.2):
    x,y=[],[]
    for file in glob.glob('/content/drive/My Drive/archive/Actor_*/*.wav'):
        file_name=os.path.basename(file)
        emotion=emotions[file_name.split("-")[2]]
        if emotion not in observed_emotions:
            continue
        feature=extract_feature(file, mfcc=True, chroma=True, mel=True)
        x.append(feature)
        y.append(emotion)
    return train_test_split(np.array(x), y, test_size=test_size, train_size= 0.75,random_state=9)

In [ ]:
import time
x_train,x_test,y_train,y_test=load_data(test_size=0.25)
print((x_train.shape[0], x_test.shape[0]))


(2171, 724)


In [ ]:
print(f'Features extracted: {x_train.shape[1]}')

Features extracted: 180


In [ ]:
from sklearn.svm import SVC 
NUM_EPOCH=500
LOG_FOR_EVERY=10
for i in range(NUM_EPOCH):
  svm_model_linear = SVC(kernel = 'rbf', C = 10,gamma=0.01).fit(x_train, y_train)

svm_predictions = svm_model_linear.predict(x_test) 


print(accuracy_score(y_true=y_test,y_pred=svm_predictions))
print(classification_report(y_test,svm_predictions)) 
# creating a confusion matrix 
print(confusion_matrix(y_test, svm_predictions) )

0.8646408839779005
              precision    recall  f1-score   support

       angry       0.59      0.98      0.74        95
        calm       0.96      0.92      0.94        98
     disgust       0.92      0.88      0.90        99
     fearful       0.95      0.74      0.83       107
       happy       0.91      0.87      0.89        91
     neutral       1.00      1.00      1.00        40
         sad       0.91      0.75      0.82       105
   surprised       0.98      0.89      0.93        89

    accuracy                           0.86       724
   macro avg       0.90      0.88      0.88       724
weighted avg       0.89      0.86      0.87       724

[[93  0  0  2  0  0  0  0]
 [ 4 90  2  0  0  0  2  0]
 [ 8  0 87  0  2  0  2  0]
 [18  0  2 79  4  0  2  2]
 [10  0  0  2 79  0  0  0]
 [ 0  0  0  0  0 40  0  0]
 [18  4  2  0  2  0 79  0]
 [ 6  0  2  0  0  0  2 79]]


In [ ]:
from sklearn.ensemble import RandomForestClassifier
  
 # create regressor object 
classifier = RandomForestClassifier(n_estimators = 10, random_state = 0)
NUM_EPOCH=500
LOG_FOR_EVERY=10
for i in range(NUM_EPOCH): 
  classifier.fit(x_train, y_train) 
  
  

c_p = classifier.predict(x_test) 


print(accuracy_score(y_true=y_test,y_pred=c_p))
print(classification_report(y_test,c_p)) 
# creating a confusion matrix 
print(confusion_matrix(y_test,c_p) )

0.8895027624309392
              precision    recall  f1-score   support

       angry       0.95      0.99      0.97        95
        calm       0.83      0.91      0.87        98
     disgust       0.84      0.85      0.84        99
     fearful       0.92      0.92      0.92       107
       happy       0.89      0.91      0.90        91
     neutral       0.95      0.88      0.91        40
         sad       0.87      0.83      0.85       105
   surprised       0.90      0.83      0.87        89

    accuracy                           0.89       724
   macro avg       0.89      0.89      0.89       724
weighted avg       0.89      0.89      0.89       724

[[94  0  0  0  1  0  0  0]
 [ 0 89  2  0  0  0  7  0]
 [ 0  5 84  2  4  2  0  2]
 [ 1  0  4 98  2  0  2  0]
 [ 2  0  2  2 83  0  0  2]
 [ 0  3  0  0  2 35  0  0]
 [ 2  8  2  2  0  0 87  4]
 [ 0  2  6  2  1  0  4 74]]


In [ ]:
from sklearn.neighbors import KNeighborsClassifier

model = KNeighborsClassifier(n_neighbors=5,weights='distance',p=5)

NUM_EPOCH=500
LOG_FOR_EVERY=10
for i in range(NUM_EPOCH):
  model.fit(x_train,y_train)


#Predict Output
predicted= model.predict(x_test) 
print(accuracy_score(y_true=y_test,y_pred= predicted))
print(classification_report(y_test,predicted)) 
# creating a confusion matrix 
print(confusion_matrix(y_test,predicted) )

0.8729281767955801
              precision    recall  f1-score   support

       angry       0.90      0.96      0.93        95
        calm       0.85      0.94      0.89        98
     disgust       0.93      0.84      0.88        99
     fearful       0.90      0.81      0.85       107
       happy       0.87      0.91      0.89        91
     neutral       0.72      0.90      0.80        40
         sad       0.91      0.81      0.86       105
   surprised       0.82      0.84      0.83        89

    accuracy                           0.87       724
   macro avg       0.86      0.88      0.87       724
weighted avg       0.88      0.87      0.87       724

[[91  0  0  2  2  0  0  0]
 [ 0 92  2  0  0  2  2  0]
 [ 0  4 83  2  2  0  2  6]
 [ 4  0  4 87  6  0  0  6]
 [ 2  0  0  4 83  0  0  2]
 [ 0  2  0  0  0 36  2  0]
 [ 0  6  0  2  2  8 85  2]
 [ 4  4  0  0  0  4  2 75]]
